In [5]:
!pip install beautifulsoup4
!pip install chromadb

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  ERROR: Command errored out with exit status 1:
   command: 'c:\python38\python.exe' 'c:\python38\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\elias\AppData\Local\Temp\tmpbgn2d4yq'
       cwd: C:\Users\elias\AppData\Local\Temp\pip-install-n88io0s8\bcrypt
  Complete output (57 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-38
  creating build\lib.win-amd64-cpython-38\bcrypt
  copying src\bcrypt\__init__.py -> build\lib.win-amd64-cpython-38\bcrypt
  running egg_info
  writing src\bcrypt.egg-info\PKG-INFO
  writing dependency_links to src\bcrypt.egg-info\dependency_links.txt
  writing requirements to src\bcrypt.egg-info\requires.txt
  writing top-level names to src\bcrypt.egg-info\top_level.txt
  reading manifest file 'src\bcrypt.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE'
  writing manifest file 'src\bcrypt.egg-info\SOURCES.txt'
  co


  Using cached https://files.pythonhosted.org/packages/cc/63/b7d76109331318423f9cfb89bd89c99e19f5d0b47a5105439a629224d297/chromadb-0.4.24-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/9c/96b321bf89aac1601ffc9cd30c434830d3af659c0fde9f1cec9c2e62e02f/graphlib_backport-1.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6f/34/164e57fec8a9693d7e6ae2d1a345482020ea9e9b32eab95a90bb3eaea83d/kubernetes-29.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/40/4b4995eb79aa71a9f373cde713c1a03d03ce5e3b97810882e16d109787de/mmh3-4.1.0-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/21/26/a5b30d5d10c187495ee6e99d814b9b76667aa1e66b8f19a8eac1a5352b7c/pulsar_client-3.4.0-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.o

In [6]:
from langchain.docstore.document import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from dotenv import load_dotenv

In [7]:

load_dotenv()

apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://owasp.org/Top10/"}], "maxCrawlPages": 10, "crawlerType": "cheerio"},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
index = VectorstoreIndexCreator().from_loaders([loader])



ImportError: Could not import apify-client Python package. Please install it with `pip install apify-client`.

In [ ]:
# Query the vector store
query = "What is the first item in the OWASP Top 10?"
result = index.query(query)
print(result)